In [1]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

# Data transformations
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(256, scale=(0.8, 1.0)),
    transforms.RandomRotation(15),
    transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_tfms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [2]:
data_dir = '.'  # current directory containing train/valid/test
batch_size = 32

train_ds = datasets.ImageFolder(data_dir + '/train', transform=train_tfms)
valid_ds = datasets.ImageFolder(data_dir + '/valid', transform=test_tfms)
test_ds = datasets.ImageFolder(data_dir + '/test', transform=test_tfms)

trainloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
validloader = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)
testloader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

print(f"Number of training images: {len(train_ds)}")
print(f"Number of validation images: {len(valid_ds)}")
print(f"Number of test images: {len(test_ds)}")
print(f"Number of classes: {len(train_ds.classes)}")


Number of training images: 4543
Number of validation images: 2241
Number of test images: 2360
Number of classes: 102


In [3]:
model = models.vgg16(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

n_inputs = model.classifier[6].in_features
n_classes = len(train_ds.classes)

model.classifier[6] = nn.Sequential(
    nn.Linear(n_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, n_classes),
    nn.LogSoftmax(dim=1)
)

# Enable gradients for classifier parameters
for param in model.classifier[6].parameters():
    param.requires_grad = True

print(model)


c:\Users\racha\anaconda3\envs\pythonProject3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\racha\anaconda3\envs\pythonProject3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier[6].parameters())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"Using device: {device}")


Using device: cuda


In [5]:
n_epochs = 10

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(trainloader.dataset)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in validloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total

    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss:.4f}, Val Accuracy: {val_acc:.4f}")


Epoch 1/10, Loss: 2.0356, Val Accuracy: 0.8010
Epoch 2/10, Loss: 0.9424, Val Accuracy: 0.8380
Epoch 3/10, Loss: 0.7261, Val Accuracy: 0.8327
Epoch 4/10, Loss: 0.6709, Val Accuracy: 0.8487
Epoch 5/10, Loss: 0.6031, Val Accuracy: 0.8291
Epoch 6/10, Loss: 0.5563, Val Accuracy: 0.8407
Epoch 7/10, Loss: 0.5166, Val Accuracy: 0.8617
Epoch 8/10, Loss: 0.4889, Val Accuracy: 0.8465
Epoch 9/10, Loss: 0.5000, Val Accuracy: 0.8407
Epoch 10/10, Loss: 0.4371, Val Accuracy: 0.8581


In [6]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_acc = correct / total
print(f"Test Accuracy: {test_acc:.4f}")


Test Accuracy: 0.8398
